# 분산 분석
- section 01 : 일원 분산 분석
- section 02 : 이원 분산 분석

## section 01 : 일원 분산 분석
- 3개 이상의 집단 간의 평균 차이가 통계적으로 유의한지 검정하는 방법
- 집단을 나누는 요인이 하나이고 집단의 수가 3개 이상일 때 사용


### 기본 가정
- 분산 분석의 기본 가정은 독립 표본 t-검정과 매우 유사
    - 독립성 : 각 집단의 관측치들은 모든 다른 집단의 관측치들과 독립적
    - 정규성 : 각 집단에서의 관측치는 정규분포를 따른다.
    - 등분산성 : 모든 집단에서의 관측치는 동일한 분산을 가진다.

### 귀무가설과 대립가설
- 귀무가설(H0) : 모든 집단의 평균은 동일
- 대립가설(H1) : 집단의 평균에는 차이가 있다.

### 일원 분산 분석
- 사이파이의 f_oneway를 사용한다.
```python
scipy.stats.f_oneway(sample1,sample2,sample3,...)
``` 
- smaple1,2,3 : 집단 데이터


In [2]:
# 문제
# 데이터는 4종류의 비료를 사용한 식물의 성장에 대한 실험 결과이다.
# 이 실험에서는 비슷한 조건의 식물 40개를 무작위로 10개씩 나누고 화학 비료 A,B,C,D를 일정 기간 사용한 후 성장량을 측정했다.
# 성장의 차이가 있는지 유의수준 0.05하에서 검정

# 귀무가설 : 네 가지 비료의 효과는 동일하다.
# 대립가설 : 비료의 효과에는 차이가 있다.

import pandas as pd

df = pd.DataFrame({
    'A':[10.5,11.3,10.8,9.6,11.1,10.2,10.9,11.4,10.5,10.3],
    'B':[11.9,12.4,12.1,13.2,12.5,11.8,12.2,12.9,12.4,12.3],
    'C':[11.2,11.7,11.6,10.9,11.3,11.1,10.8,11.5,11.4,11.0],
    'D':[9.8,9.4,9.1,9.5,9.6,9.9,9.2,9.7,9.3,9.4]
})

df.head(2)

,A,B,C,D
0,10.5,11.9,11.2,9.8
1,11.3,12.4,11.7,9.4


- 정규성 검정 : A,B,C,D의 모든 그룹에 대한 p-value가 0.05보다 크므로 모든 그룹의 데이터가 정규분포를 따름.
- 등분산성 검정 : p-value가 0.05보다 크므로 모든 그룹의 분산이 동일하다고 볼 수 있다.
- 일원 분산 분석 : p-value가 0.05보다 작으므로 적어도 두 그룹 간의 평균 성장량에는 차이가 있다고 볼 수 있다.

In [3]:
from scipy import stats

print('=== 정규성 검정 ===')
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))

print('\n=== 등분산성 검정 ===')
print(stats.levene(df['A'],df['B'],df['C'],df['D']))

print('\n=== 일원 분산 분석')
print(stats.f_oneway(df['A'],df['B'],df['C'],df['D']))


=== 정규성 검정 ===
ShapiroResult(statistic=np.float64(0.9649054066073819), pvalue=np.float64(0.840016154346871))
ShapiroResult(statistic=np.float64(0.9468040874196029), pvalue=np.float64(0.6308700692815115))
ShapiroResult(statistic=np.float64(0.9701646110856058), pvalue=np.float64(0.8923673061902999))
ShapiroResult(statistic=np.float64(0.9752339025839641), pvalue=np.float64(0.9346854448707635))

=== 등분산성 검정 ===
LeveneResult(statistic=np.float64(1.9355354288758708), pvalue=np.float64(0.14127835331346628))

=== 일원 분산 분석
F_onewayResult(statistic=np.float64(89.12613851177174), pvalue=np.float64(1.001838152252373e-16))


### 일원 분산 분석(ols 활용)
- 일원 분산 분석 방법에는 사이파이의 f_oneway와 스태츠모델즈의 ols,anova_lmd을 사용하는 방법이 있다.
- ols() 함수는 R 스타일의 표현식을 사용한다.


<table>
  <tr>
    <td>구분</td>
    <td>설명</td>
  </tr>
  <tr>
    <td rowspan="2">물결(~)</td>
    <td>종속변수와 독립 변수를 구분하는 기호</td>
    
  </tr>
  <tr>
  <td>target~a에서 target은 종속변수고, a는 독립변수임을 나타낸다.</td>
  </tr>
    <tr>
    <td rowspan="2">더하기(+)</td>
    <td>여러 독립변수들을 모델에 포함하기 위한 기호다.</td>
    
  </tr>
  <tr>
  <td>target~a+b+c+d 는 a,b,c,d 독립변수를 모두 포함하겠다는 의미.</td>
  </tr>
</table>

In [6]:
# !pip install statsmodels
!pip install patsy

In [9]:
import pandas as pd
df = pd.read_csv('./ch2_data/fertilizer.csv')

In [10]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('성장~C(비료)',df).fit()
print(anova_lm(model))

            df    sum_sq    mean_sq          F        PR(>F)
C(비료)      3.0  43.21875  14.406250  89.126139  1.001838e-16
Residual  36.0   5.81900   0.161639        NaN           NaN


<table>
  <tr>
    <td>구분</td>
    <td>설명</td>
  </tr>
  <tr>
    <td rowspan="4">df(자유도)</td>
    <td>전체 데이터 관찰 수 : 4*10 = 40</td>
  </tr>
  <tr>
  <td>전체 데이터(관찰) 수의 자유도 : 40 -1 = 39</td>
  </tr>
    <tr>
  <td>그룹(처리조건)의 자유도: 4 - 1 = 3 </td>
  </tr>
    <tr>
  <td>잔차의 자유도 : 39 -3 = 36</td>
  </tr>
    <tr>
    <td rowspan="1">sum_sq(제곱합)</td>
    <td>그룹(처리조건)별 또는 잔차별 제곱합</td>
  </tr>
   <tr>
    <td rowspan="1">mean_sq(평균 제곱)</td>
    <td>평균 제곱합으로 sum_sq를 df로 나눈 값</td>
  </tr>
  <tr>
    <td rowspan="1">F(F-통계량)</td>
    <td>F-통계량</td>
  </tr>
  <tr>
    <td rowspan="1">PR(>F)(p-값)</td>
    <td>F-통계량에 대한 p-value</td>
  </tr>
</table>

## section 02 : 이원 분산 분석
- 이원 분산 분석(Two-way ANOVA)은 요인의 수가 2개인 경우.
<table>
  <tr>
    <td>A 요인 : 학습방법</td>
    <td colspan="2">도서</td>
    <td colspan="2">강의</td>
  </tr>
  <tr>
    <td>B 요인 : 학습 장소</td>
    <td>집</td>
    <td>카페</td>
    <td>집</td>
    <td>카페</td>
  </tr>
</table>

### 기본 가정
- 독립성 : 각 집단의 관측치들은 모든 다른 집단의 관측치들과 독립적이다.(기본 가정)
- 정규성 : 각 집단에서의 관측치는 정규 분포를 따른다. (Shapiro-Wilk test)
- 등분산성 : 모든 집단에서의 관측치는 동일한 분산을 가진다(Levene test)

### 귀무가설과 대립가설
- 이원 분산 분석은 주 효과뿐만 아니라 상호작용 효과에도 관심을 둔다.
- 학습 방법과 학습 장소를 요인으로 이원 분산 분석을 실시할 경우 주 효과 2개, 상호작용 효과 1개로 다음과 같은 가설을 세울 수 있음.

- 주 효과 : A 요인(학습방법)
    - 귀무가설 : 학습 방법에 따라 성적 차이가 없다.
    - 대립가설 : 학습 방법에 따라 성척 차이가 있다.
- 주 효과 : B 요인(학습장소)
    - 귀무가설 : 학습 장소에 따라 성적 차이가 없다.
    - 대립가설 : 학습 장소에 따라 성척 차이가 있다.

- 상호 작용 효과
    - 귀무가설 : A 요인과 B 요인 간에 따라 상호작용이 없다.
    - 대립가설 : A 요인과 B 요인 간에 따라 상호작용이 있다.
---

### 이원 분산 분석
- 스테츠모델즈(statsmodels)의 ols와 anova_lm을 사용

```python
model = ols(종속변수 ~ C(요인1) +C(요인2) + C(요인1):C(요인2)).fit()
anova_lm(model,typ='숫자')
``` 
    - 종속 번수 : 연속형 변수
    - 요인 1 : 첫 번째 독립변수(범주형)
    - 요인 2 : 두 번째 독립변수(범주형)
    - typ=1 : 변수의 순서에 따른 분석(기본)
    - typ=2 : 각 변수의 독립적인 효과 분석
    - typ=3 : 모든 변수와 상호작용을 동시에 고려해 분석


#### 문제
- 데이터는 네 가지 종류의 나무(A,B,C,D)에 대해 세 가지 종류의 비료(1,2,3)를 사용해 성장률을 조사한 결과
- 비교 간 및 나무 종류 간의 성장률 차이가 있는지 유의수진 0.05하에서 검정
- 나무(주 효과)
    - 귀무가설 : 모든 나무 종류의 성장률은 동일
    - 대립가설 : 나무 종류의 성장률에는 차이가 있음.
- 비료(주 효과)
    - 귀무가설 : 모든 비료의 성장률 효과는 동일
    - 대립가설 : 비료의 성장률 효과에는 차이가 있음.
- 상호작용 효과
    - 귀무가설 : 나무 종류와 비료 간의 상호작용은 성장률에 영향을 주지 않는다.
    - 대립가설 : 나무 종류와 비료 간에는 성장률에 영향을 주는 상호작용이 있다.
    

In [2]:
# 문제
# 데이터는 네 가지 종류의 나무(A,B,C,D)에 대해 세 가지 종류의 비료(1,2,3)를 사용해 성장률을 조사한 결과
# 비교 간 및 나무 종류 간의 성장률 차이가 있는지 유의수진 0.05하에서 검정

import pandas as pd
df = pd.read_csv("./ch2_data/tree.csv")
print(df.sample(10))

    나무  비료        성장률
100  D   2  53.846293
84   C   3  57.915064
8    A   1  45.305256
71   C   2  78.380366
93   D   1  61.723379
54   B   3  71.309995
101  D   2  63.793547
1    A   1  48.617357
112  D   3  71.602302
115  D   3  74.015473


#### ols(종속 변수 ~ 요인 1 + 요인2 + 요인1:요인2).fit()
- ols(최소제곱 선형 회귀) 모델을 만들고 데이터를 넣어서 학습.
- 성장률이 나무, 비료 그리고 나무와 비료의 상호작용에 어떻게 영향을 받는지 모델링

##### 1. sm.stats.anova_lm(model)
- sm.stats.anova_lm(model,typ={1,2,3})으로 ANOVA 테이블을 만든다.
- typ 파라미터는 ANOVA 테이블을 게산하는 방식을 결정한다.
    - typ=1 : 순차적 합곱제곱(SS), 변수를 하나씩 추가하면서 그 변수가 결과에 얼마나 영향을 미치는지 순서대로 계산
    - typ=2 : 불순차적 합곱제곱(SS), 각 변수가 모델 전체에 독립적으로 얼마나 영향을 미치는지
    - typ=3 : 타입3 합곱제곱 , 변수들 간의 상호작용까지 고려해 모든 변수가 모델 안에 있는 상태에서 각 변수의 효과를 계산
    

In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('성장률 ~ 나무 + 비료 + 나무:비료', data=df).fit()
anova_table = sm.stats.anova_lm(model)
print(anova_table)

             df       sum_sq      mean_sq          F        PR(>F)
나무          3.0  4783.353938  1594.451313  18.391274  9.016693e-10
비료          1.0   873.322002   873.322002  10.073374  1.942421e-03
나무:비료       3.0   394.801585   131.600528   1.517952  2.137666e-01
Residual  112.0  9709.960792    86.696078        NaN           NaN


- 나무는 A,B,C,D의 범주로 구성되어있지만, 비료는 1,2,3과 같은 숫자로 표현되어있음.
- 연속형 변수로 잘못 해석될 위험이 있으므로 범주형 변수를 분석할 때는 C()를 사용해 명확히 범주형으로 처리해야한다.


In [6]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('성장률 ~ C(나무) + C(비료) + C(나무):C(비료)',data=df).fit()
anova_table = sm.stats.anova_lm(model)
print(anova_table)

                df       sum_sq      mean_sq          F        PR(>F)
C(나무)          3.0  4783.353938  1594.451313  18.855528  6.600012e-10
C(비료)          2.0  1127.924259   563.962129   6.669256  1.857612e-03
C(나무):C(비료)    6.0   717.520672   119.586779   1.414199  2.157357e-01
Residual     108.0  9132.639448    84.561476        NaN           NaN


##### 나무 종류에 대한 효과
- df : 자유도는 3이다. 4개의 나무 종류가 있으므로 4-1 = 3으로 계산
- sum_sq : 총 제곱합은 4783.353938이다.
- mean_sq : 평균 제곱은 1594.451313이다.
- F : F-통계량은 18.855528이다.
- PR(>F) : p-value(값)은 6.600012e-10이다.
- 결론 : 5%의 유의수준에서 p-value가 0.05보다 작으므로 나무 종류에 따른 성장률에 유의미한 차이가 있다고 함.



##### 비료에 대한 효과
- df : 자유도는 2이다. 3개의 비료 종류가 있으므로 3-1 = 2 으로 계산
- sum_sq : 총 제곱합은 1127.924259 이다.
- mean_sq : 평균 제곱은 563.962129 이다.
- F : F-통계량은 6.669256 이다.
- PR(>F) : p-value(값)은 1.857612e-03 이다.
- 결론 : 5%의 유의수준에서 p-value가 0.05보다 작으므로 비료의 종류에 따른 성장률에 유의미한 차이가 있다고 함.


##### 나무와 비료 간의 상호작용 효과
- df : 자유도는 6이다 (4(나무종류)*3(비료종류)- 4(나무 종류)-3(비료종류)+1) = 6 
- sum_sq : 총 제곱합은 717.520672 이다.
- mean_sq : 평균 제곱은 119.586779 이다.
- F : F-통계량은 1.414199 이다.
- PR(>F) : p-value(값)은 2.157357e-01 이다.
- 결론 : 5%의 유의수준에서 p-value가 0.05보다 크므로 나무 종류와 비료 종류 간의 상호작용은 성장률에 유의미한 영향을 주지 않는다.


##### 최종 결론
- 나무 종류와 비료 종류는 모두 성장률에 유의미한 영향을 준다.
- 나무와 비료 간의 상호작용은 성장률에 유의미한 영향을 주지 않는다.
